## Init e dependencies

In [4]:
! pip install langchain==0.2.7 langchain_aws langchain-community langchain_core boto3 botocore
! pip install dotenv

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
  Using cached dotenv-0.0.5.tar.gz (2.4 kB)
  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [64 lines of output]
      /usr/lib/python3/dist-packages/setuptools/installer.py:27: SetuptoolsDeprecationWarning: setuptools.installer is deprecated. Requirements should be satisfied by a PEP 517 installer.
        warnings.warn(
        error: subprocess-exited-with-error
      
        × python setup.py egg_info did not run successfully.
        │ exit code: 1
        ╰─> [16 lines of output]
            Traceback (most recent call last):
              File "<string>", line 2, in <module>
              File "<pip-setuptools-caller>", line 14, in <module>
              File "/tmp/pip-wheel-uk4lutt1/distribute_4b3b5d64e2394d728ec

In [5]:
from dotenv import load_dotenv
load_dotenv()
import os

In [ ]:
### llm endpoints

In [ ]:
import boto3
from langchain_aws import ChatBedrock
from botocore.config import Config
import urllib3
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

bedrock_client = boto3.client(
        service_name='bedrock-runtime',
        region_name='eu-west-1',
        aws_access_key_id=os.getenv('AWS_ACCESS_KEY_ID'),
        aws_secret_access_key=os.getenv('AWS_SECRET_ACCESS_KEY'),
        verify=False,  # Disable SSL verification
        config=Config(
            proxies={'https': None}
        )
    )

llm = ChatBedrock(
        model_id="anthropic.claude-3-haiku-20240307-v1:0",
        client=bedrock_client,
        model_kwargs={
            "temperature": 0,
            "max_tokens": 2000,
        }
    )


#### Cos'è un while True?

Il costrutto while True è una struttura di controllo utilizzata nei linguaggi di programmazione per creare loop (cicli) infiniti. In Python, un ciclo while esegue ripetutamente un blocco di codice finché la condizione specificata è vera. Nel caso di while True, la condizione è sempre vera, quindi il ciclo continua all'infinito finché non viene interrotto manualmente.

In questo caso, il ciclo viene utilizzato per richiedere all'utente (tramite comando "input") un messaggio da girare alla LLM

In [2]:
print("Chatbot per l'ordine - Digita 'esci' per uscire")
while True:
    comando = input("Scrivi 'esci' per terminare: ")
    if comando.lower() == "esci":
        print("Ciclo terminato.")
        break

print("Fuori dal ciclo")

Chatbot per l'ordine - Digita 'esci' per uscire
Ciclo terminato.
Fuori dal ciclo


esempio con flusso semplice

tracce:
- sistema di ordini
- altre idee?

ordini

cahoot: 
- gradimento
- quali istruttori hanno preferito
- discussione aperta sui temi trattai: quale piace di più, qualcosa da approfondire, da rivedere
- è piaciuto l'approccio?
